In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train = pd.read_csv('../data/processed/train.csv')
print('shape: ', train.shape)
print('columns', train.columns)

shape:  (20631, 68)
columns Index(['id', 'cycle', 'setting1', 'setting2', 's1', 's2', 's3', 's4', 's5',
       's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       's17', 's20', 's21', 'RUL', 'label1', 'label2', 'avg_setting1',
       'std_setting1', 'avg_setting2', 'std_setting2', 'avg_s1', 'std_s1',
       'avg_s2', 'std_s2', 'avg_s3', 'std_s3', 'avg_s4', 'std_s4', 'avg_s5',
       'std_s5', 'avg_s6', 'std_s6', 'avg_s7', 'std_s7', 'avg_s8', 'std_s8',
       'avg_s9', 'std_s9', 'avg_s10', 'std_s10', 'avg_s11', 'std_s11',
       'avg_s12', 'std_s12', 'avg_s13', 'std_s13', 'avg_s14', 'std_s14',
       'avg_s15', 'std_s15', 'avg_s16', 'std_s16', 'avg_s17', 'std_s17',
       'avg_s20', 'std_s20', 'avg_s21', 'std_s21'],
      dtype='object')


In [3]:
test = pd.read_csv('../data/processed/test.csv')
print('shape: ', test.shape)
print('columns', test.columns)

shape:  (13096, 68)
columns Index(['id', 'cycle', 'setting1', 'setting2', 's1', 's2', 's3', 's4', 's5',
       's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       's17', 's20', 's21', 'RUL', 'label1', 'label2', 'avg_setting1',
       'std_setting1', 'avg_setting2', 'std_setting2', 'avg_s1', 'std_s1',
       'avg_s2', 'std_s2', 'avg_s3', 'std_s3', 'avg_s4', 'std_s4', 'avg_s5',
       'std_s5', 'avg_s6', 'std_s6', 'avg_s7', 'std_s7', 'avg_s8', 'std_s8',
       'avg_s9', 'std_s9', 'avg_s10', 'std_s10', 'avg_s11', 'std_s11',
       'avg_s12', 'std_s12', 'avg_s13', 'std_s13', 'avg_s14', 'std_s14',
       'avg_s15', 'std_s15', 'avg_s16', 'std_s16', 'avg_s17', 'std_s17',
       'avg_s20', 'std_s20', 'avg_s21', 'std_s21'],
      dtype='object')
